In [ ]:
"""
This notebook conducts PCA on extracted feature data and
produces scatter plots based on rated valence
"""

In [ ]:
%matplotlib widget

In [ ]:
import glob
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import torch
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, normalize

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
plt.style.use('ggplot')

In [ ]:
ROOT_FOLDER = ""
FEAT_FOLDER = f"{ROOT_FOLDER}\Extracted_Features"
SAVE_FOLDER = f"{ROOT_FOLDER}\Face_Feature_Data"
VIS_RATINGS = f"{ROOT_FOLDER}\SH09_avoidance_Ratings_vis.xlsx"
APEX_FRAMES = f"{ROOT_FOLDER}Apex_frame_data.txt"
NUM_RATINGS = 9
NUM_TRIALS = 36

# Set number of  PCA components
PCA_COMP_NO = 10

In [ ]:
# Make scatter plot of PCA components grouped by rating
def plot_pca(data, labels, targets, target_names=None, num_dims=2, alpha_values=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], title=None):

    # If labels not provided, use target names (ratings)
    if not target_names:
        target_names = [str(x) for x in targets]


    fig, ax = plt.subplots()
    if not title:
        ax.set_title("principal components of AU activation probabilities")
    else:
        ax.set_title(title)
    ax.set_xlabel("Principal Component 1")
    ax.set_ylabel("Principal Component 2")
    lines = [None for x in targets]

    # Plots can be 2d or 3d
    if num_dims == 2:
        for idx, target in enumerate(targets):
            lines[idx]= ax.scatter(
                data[labels == target, 0],
                data[labels == target, 1],
                alpha=alpha_values[idx],
                label=target_names[idx],
            )
    elif num_dims == 3:
        ax = fig.add_subplot(projection="3d")
        for idx, target in enumerate(targets):
            lines[idx] = ax.scatter(
                data[labels == target, 0],
                data[labels == target, 1],
                data[labels == target, 2],
                alpha=alpha_values[idx],
                label=target_names[idx],
            )
        ax.set_zlabel("Principal Component 3")
    leg = ax.legend(fancybox=True, scatterpoints=1)
    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.legendHandles, lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    # For interactive legend
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend, so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)

    plt.show()


In [ ]:
ratings = pd.read_excel(VIS_RATINGS, header=0)
apex_df = pd.read_csv(APEX_FRAMES, sep="\s+", header=0)
apex_num = len(apex_df.index)

# Our model detects 41 AUs
features = np.zeros((apex_num, 41))
print(features.shape)
features_count = 0

p_ratings = np.zeros(apex_num)

experiments = glob.glob(FEAT_FOLDER + "/*/")
num_experiments = len(experiments)
print(num_experiments)

# Loop through all experiments 
for experiment_path in experiments:
    experiment = os.path.basename(os.path.normpath(experiment_path))
    print(experiment)
    feature_paths = glob.glob(experiment_path + "*.npy")

    # For each trial find corresponding rating and extracted feature array
    for feature_path in feature_paths:
        file_name = os.path.basename(os.path.normpath(feature_path))
        block_num = int(file_name.split("_")[1])
        trial_num = int(re.split("[_.]",file_name)[3])
        p_rating = ratings[int(experiment)][(block_num-1)*NUM_TRIALS + (trial_num-1)]
        if np.isnan(p_rating):
            continue
        feature_path = f"{experiment_path}\Block_{block_num}_Trial_{trial_num}.npy"
        with open(feature_path,"rb") as f:
            extracted_feature = np.load(f, allow_pickle=True)
        features[features_count, :] = extracted_feature
        p_ratings[features_count] = p_rating            
        features_count += 1


In [ ]:
# Remove rows with invalid valence rating
rows_to_remove = np.where(p_ratings == 0)[0]
features = np.delete(features, rows_to_remove, axis=0)
p_ratings = np.delete(p_ratings, rows_to_remove, axis=0)
print(features.shape)
print(np.mean(features, axis=0),np.std(features, axis=0))

In [ ]:
# Get 10 most important principal components and save them
pca = PCA(n_components=PCA_COMP_NO, svd_solver="full")
X_r = pca.fit(features).transform(features)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))
with open(f'{SAVE_FOLDER}pca_{PCA_COMP_NO}_comps_2.npy', 'wb') as f:
    np.save(f, X_r)

In [ ]:
# Plot and print proportion of explained variance per principal component

print ("Proportion of Variance Explained : ", pca.explained_variance_ratio_)  
out_sum = np.cumsum(pca.explained_variance_ratio_)  
print ("Cumulative Prop. Variance Explained: ", out_sum)
print(pca.explained_variance_) 

PC_values = np.arange(pca.n_components_) + 1

fig = plt.subplot(1, 2, 1)
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', color='#F8766D', linewidth=2)

plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')

fig = plt.subplot(1, 2, 2)
plt.plot(PC_values, out_sum, 'o-', color='#619CFF', linewidth=2)

plt.title('Cumulative Scree Plot')
plt.xlabel('No. of Principal Components')
plt.ylabel('Cumulative Proportion of Variance Explained')
plt.suptitle("PCA of 256x20x20 feature vectors")
plt.show()


In [ ]:
plt.style.use('seaborn-dark-palette')

In [ ]:
# 2D PC scatter plot
target_names = ["Valence 1", "Valence 5", "Valence 9"]
plot_pca(data=X_r,
         labels=p_ratings,
         targets=[1, 5, 9],
         alpha_values = [0.5, 0.1, 0.5],
         target_names=target_names)

In [ ]:
# 3D PC scatter plot
plot_pca(
    data=X_r,
    labels=p_ratings,
    targets=[1, 5, 9],
    num_dims=3,
    target_names=["Valence 1", "Valence 5", "Valence 9"],
    alpha_values=[0.5, 0.05, 0.5]
)

In [ ]:
# 2D PC scatter plot for all 9 ratings
plot_pca(data=X_r, labels=p_ratings, targets=[x for x in range(1,10)])
